In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np

In [2]:
TRAIN_TEST_SPLIT = 0.7
IM_WIDTH, IM_HEIGHT = 133, 225

In [10]:
from keras.utils import to_categorical
from PIL import Image
class DataGenerator():
    def __init__(self, df):
        self.df = df
        
    def generate_split_indexes(self):
        p = np.random.permutation(len(self.df))
        train_up_to = int(len(self.df) * TRAIN_TEST_SPLIT)
        train_idx = p[:train_up_to]
        test_idx = p[train_up_to:]
        train_up_to = int(train_up_to * TRAIN_TEST_SPLIT)
        train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]
        
        return train_idx, valid_idx, test_idx
        
    def preprocess_image(self, img_path):
        im = Image.open(img_path)
        im = im.resize((IM_WIDTH, IM_HEIGHT))
        im = np.array(im) / 255.0
        
        return im
        
    
    def generate_images(self, image_idx, is_training, batch_size=16):
        
        # arrays to store batched data
        images, necks, lengths, patterns = [], [], [], []
        while True:
            for idx in image_idx:
                person = self.df.iloc[idx]
                
                neck = person['neck']
                length = person['sleeve_length']
                pattern = person['pattern']
                file = 'imgs_edge/'+person['filename']
            
                necks.append(to_categorical(neck))
                patterns.append(to_categorical(pattern))
                lengths.append(to_categorical(length))
                
                im = self.preprocess_image(file)
                images.append(im)
                
                # yielding condition
                if len(images) >= batch_size:
                    yield np.array(images), [np.array(necks), np.array(lengths), np.array(patterns)]
                    images, necks, lengths, patterns = [], [], [], []
                    
            if not is_training:
                break
            

In [11]:
df = pd.read_csv('cleandata.csv', index_col=0)       
for ix, fname in zip(range(len(df['filename'])), df['filename']):
    if os.path.exists('imgs_edge/'+fname):
        pass
    else: 
        df = df[df.filename != fname]
print(df.shape)
data_generator = DataGenerator(df)
train_idx, valid_idx, test_idx = data_generator.generate_split_indexes() 

(1797, 4)


In [12]:
num_necks, num_lengths, num_patterns = 7, 4, 10

In [13]:
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Lambda
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
import tensorflow as tf
class MultiOutputModel():

    def make_default_hidden_layers(self, inputs):
        
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(32, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        x = Dropout(0.25)(x)
        return x
    def build_neck_branch(self, inputs, num_neck):
        
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_necks)(x)
        x = Activation("softmax", name="neck_output")(x)
        return x
    def build_length_branch(self, inputs, num_length):
        
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_lengths)(x)
        x = Activation("softmax", name="length_output")(x)
        return x
    def build_pattern_branch(self, inputs, num_pattern):   
        
        x = self.make_default_hidden_layers(inputs)
        x = Flatten()(x)
        x = Dense(128)(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(num_patterns)(x)
        x = Activation("softmax", name="pattern_output")(x)
        return x
    def assemble_full_model(self, width, height, num_necks, num_lengths, num_patterns):
        
        input_shape = (height, width, 3)
        inputs = Input(shape=input_shape)
        neck_branch = self.build_neck_branch(inputs, num_necks)
        length_branch = self.build_length_branch(inputs, num_lengths)
        pattern_branch = self.build_pattern_branch(inputs, num_patterns)
        model = Model(inputs=inputs,
                     outputs = [neck_branch, length_branch, pattern_branch],
                     name="Shape_net")
        return model
    
model = MultiOutputModel().assemble_full_model(IM_WIDTH, IM_HEIGHT, num_necks, num_lengths, num_patterns)

NotImplementedError: Cannot convert a symbolic Tensor (neck_output_1/Identity:0) to a numpy array.

In [ ]:
from keras.optimizers import Adam
init_lr = 1e-4
epochs = 100
opt = Adam(lr=init_lr, decay=init_lr / epochs)
model.compile(optimizer=opt, 
              loss={
                  'neck_output': 'categorical_crossentropy', 
                  'length_output': 'categorical_crossentropy', 
                  'pattern_output': 'categorical_crossentropy'},
              loss_weights={
                  'neck_output': 1, 
                  'length_output': 1, 
                  'pattern_output': 1},
              metrics={
                  'neck_output': 'accuracy', 
                  'length_output': 'accuracy',
                  'pattern_output': 'accuracy'})

In [19]:
from keras.callbacks import ModelCheckpoint
batch_size = 32
valid_batch_size = 32
train_gen = data_generator.generate_images(train_idx, is_training=True, batch_size=batch_size)
valid_gen = data_generator.generate_images(valid_idx, is_training=True, batch_size=valid_batch_size)
callbacks = [
    ModelCheckpoint("./model_checkpoint", monitor='val_loss')
]
history = model.fit_generator(train_gen,
                    steps_per_epoch=len(train_idx)//batch_size,
                    epochs=epochs,
                    callbacks=callbacks,
                    validation_data=valid_gen,
                    validation_steps=len(valid_idx)//valid_batch_size)

<ipython-input-10-457115c27423>:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  yield np.array(images), np.asarray([np.array(necks), np.array(lengths), np.array(patterns)])


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [18]:
import plotly.graph_objects as go
plt.clf()
fig = go.Figure()
fig.add_trace(go.Scatter(
                    y=history.history['neck_output_acc'],
                    name='Train'))
fig.add_trace(go.Scatter(
                    y=history.history['val_neck_output_acc'],
                    name='Valid'))
fig.update_layout(height=500, 
                  width=700,
                  title='Accuracy for neck feature',
                  xaxis_title='Epoch',
                  yaxis_title='Accuracy')
fig.show()

ModuleNotFoundError: No module named 'plotly'

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scattergl(
                    y=history.history['loss'],
                    name='Train'))
fig.add_trace(go.Scattergl(
                    y=history.history['val_loss'],
                    name='Valid'))
fig.update_layout(height=500, 
                  width=700,
                  title='Overall loss',
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()